# Generating captions from Flickr-30K feature embeddings
This notebook generates captions from the Flickr-30K dataset. This allows us to analyze the quality of captions generated by the LSTM we trained in the `Image_caption_train.ipynb` notebook. 

# Make sure to run the 'Image_caption_train.ipynb' notebook for at least one epoch before running this notebook.

In [7]:
import math
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

import tensorflow.python.platform
from keras.preprocessing import sequence
from collections import Counter

# Downloading Data
As mentioned in the README, in order to run this notebook, you will need VGG-16 image embeddings for the Flickr-30K dataset. These image embeddings has been provided.

Additionally, you will need the corresponding captions for these images (`results_20130124.token`).

In [8]:
model_path = './models/tensorflow'
feature_path = './data/feats.npy'
annotation_path = './data/results_20130124.token'

In [9]:
### Set Hyperparameters ###
dim_embed = 256
dim_hidden = 256
dim_in = 4096
batch_size = 1
learning_rate = 0.001
momentum = 0.9
n_epochs = 25

In [10]:
def get_data(annotation_path, feature_path):
     annotations = pd.read_table(annotation_path, sep='\t', header=None, names=['image', 'caption'])
     return np.load(feature_path,'r'), annotations['caption'].values

In [11]:
feats, captions = get_data(annotation_path, feature_path)

In [12]:
print(feats.shape)
print(captions.shape)

(158915, 4096)
(158915,)


In [13]:
print(captions[0])

Two young guys with shaggy hair look at their hands while hanging out in the yard .


In [14]:
class Caption_Generator():
    def __init__(self, dim_in, dim_embed, dim_hidden, batch_size, n_lstm_steps, n_words, init_b=None):

        self.dim_in = dim_in
        self.dim_embed = dim_embed
        self.dim_hidden = dim_hidden
        self.batch_size = batch_size
        self.n_lstm_steps = n_lstm_steps
        self.n_words = n_words
        
        # declare the variables to be used for our word embeddings
        with tf.device("/cpu:0"):
            self.word_embedding = tf.Variable(tf.random_uniform([self.n_words, self.dim_embed], -0.1, 0.1), name='word_embedding')

        self.embedding_bias = tf.Variable(tf.zeros([dim_embed]), name='embedding_bias')
        
        # declare the LSTM itself
        self.lstm = tf.contrib.rnn.BasicLSTMCell(dim_hidden)
        
        # declare the variables to be used to embed the image feature embedding to the word embedding space
        self.img_embedding = tf.Variable(tf.random_uniform([dim_in, dim_hidden], -0.1, 0.1), name='img_embedding')
        self.img_embedding_bias = tf.Variable(tf.zeros([dim_hidden]), name='img_embedding_bias')

        # declare the variables to go from an LSTM output to a word encoding output
        self.word_encoding = tf.Variable(tf.random_uniform([dim_hidden, n_words], -0.1, 0.1), name='word_encoding')
        
        # optional initialization setter for encoding bias variable 
        if init_b is not None:
            self.word_encoding_bias = tf.Variable(init_b, name='word_encoding_bias')
        else:
            self.word_encoding_bias = tf.Variable(tf.zeros([n_words]), name='word_encoding_bias')

    def build_model(self):
        # declaring the placeholders for our extracted image feature vectors, our caption, and our mask
        # (describes how long our caption is with an array of 0/1 values of length `maxlen`  
        img = tf.placeholder(tf.float32, [self.batch_size, self.dim_in])
        caption_placeholder = tf.placeholder(tf.int32, [self.batch_size, self.n_lstm_steps])
        mask = tf.placeholder(tf.float32, [self.batch_size, self.n_lstm_steps])
        
        # getting an initial LSTM embedding from our image_imbedding
        image_embedding = tf.matmul(img, self.img_embedding) + self.img_embedding_bias
        
        # setting initial state of our LSTM
        state = self.lstm.zero_state(self.batch_size, dtype=tf.float32)

        total_loss = 0.0
        with tf.variable_scope("RNN"):
            for i in range(self.n_lstm_steps): 
                if i > 0:
                   # if this isn’t the first iteration of our LSTM we need to get the word_embedding corresponding
                   # to the (i-1)th word in our caption 
                    with tf.device("/cpu:0"):
                        current_embedding = tf.nn.embedding_lookup(self.word_embedding, caption_placeholder[:,i-1]) + self.embedding_bias
                else:
                     #if this is the first iteration of our LSTM we utilize the embedded image as our input 
                    current_embedding = image_embedding
                if i > 0: 
                    # allows us to reuse the LSTM tensor variable on each iteration
                    tf.get_variable_scope().reuse_variables()

                out, state = self.lstm(current_embedding, state)

                
                if i > 0:
                    #get the one-hot representation of the next word in our caption 
                    labels = tf.expand_dims(caption_placeholder[:, i], 1)
                    ix_range=tf.range(0, self.batch_size, 1)
                    ixs = tf.expand_dims(ix_range, 1)
                    concat = tf.concat([ixs, labels],1)
                    onehot = tf.sparse_to_dense(
                            concat, tf.stack([self.batch_size, self.n_words]), 1.0, 0.0)


                    #perform a softmax classification to generate the next word in the caption
                    logit = tf.matmul(out, self.word_encoding) + self.word_encoding_bias
                    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=onehot)
                    xentropy = xentropy * mask[:,i]

                    loss = tf.reduce_sum(xentropy)
                    total_loss += loss

            total_loss = total_loss / tf.reduce_sum(mask[:,1:])
            return total_loss, img,  caption_placeholder, mask


    def build_generator(self, maxlen, batchsize=1):
        #same setup as `build_model` function 
        img = tf.placeholder(tf.float32, [self.batch_size, self.dim_in])
        image_embedding = tf.matmul(img, self.img_embedding) + self.img_embedding_bias
        state = self.lstm.zero_state(batchsize,dtype=tf.float32)

        #declare list to hold the words of our generated captions
        all_words = []
        with tf.variable_scope("RNN"):
            # in the first iteration we have no previous word, so we directly pass in the image embedding
            # and set the `previous_word` to the embedding of the start token ([0]) for the future iterations
            output, state = self.lstm(image_embedding, state)
            previous_word = tf.nn.embedding_lookup(self.word_embedding, [0]) + self.embedding_bias

            for i in range(maxlen):
                tf.get_variable_scope().reuse_variables()

                out, state = self.lstm(previous_word, state)


                # get a get maximum probability word and it's encoding from the output of the LSTM
                logit = tf.matmul(out, self.word_encoding) + self.word_encoding_bias
                best_word = tf.argmax(logit, 1)

                with tf.device("/cpu:0"):
                    # get the embedding of the best_word to use as input to the next iteration of our LSTM 
                    previous_word = tf.nn.embedding_lookup(self.word_embedding, best_word)

                previous_word += self.embedding_bias

                all_words.append(best_word)

        return img, all_words

In [15]:
if not os.path.exists('data/ixtoword.npy'):
    print ('You must run 1. O\'reilly Training.ipynb first.')
else:
    ixtoword = np.load('data/ixtoword.npy').tolist()
    n_words = len(ixtoword)
    maxlen=15
    
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    
    caption_generator = Caption_Generator(dim_in, dim_hidden, dim_embed, batch_size, maxlen+2, n_words)

    image, generated_words = caption_generator.build_generator(maxlen=maxlen)

In [23]:
def test(sess,image,generated_words,ixtoword,idx): # Naive greedy search 
    #can do beam search here?

    feats, captions = get_data(annotation_path, feature_path)
    feat = np.array([feats[idx]])
    
    saver = tf.train.Saver()
    sanity_check= False
    # sanity_check=True
    if not sanity_check:
        saved_path=tf.train.latest_checkpoint(model_path)
        saver.restore(sess, saved_path)
    else:
        tf.global_variables_initializer().run()

    generated_word_index= sess.run(generated_words, feed_dict={image:feat})
    generated_word_index = np.hstack(generated_word_index)

    generated_sentence = [ixtoword[x] for x in generated_word_index]
    print(generated_sentence)

In [27]:
test(sess,image,generated_words,ixtoword,768)

['a', 'woman', 'in', 'a', 'blue', 'dress', 'is', 'walking', 'down', 'the', 'street', '.', 'a', 'large', 'crowd']


In [5]:
# semantic analysis

import nltk
from nltk.sem import Valuation, Model
v = [('adam', 'b1'), ('betty', 'g1'), ('fido', 'd1'),
('girl', set(['g1', 'g2'])), ('boy', set(['b1', 'b2'])),
('dog', set(['d1'])),
('love', set([('b1', 'g1'), ('b2', 'g2'), ('g1', 'b1'), ('g2', 'b1')]))]
val = Valuation(v)
dom = val.domain
m = Model(dom, val)

In [6]:
# Sentiment analysis
from nltk.classify import NaiveBayesClassifier
nltk.download('subjectivity')
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)
(100, 100)


[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.


(100, 100)

In [9]:

# First, you're going to need to import wordnet: 
nltk.download('wordnet')

# Then, we're going to use the term "program" to find synsets like so: 
syns = wordnet.synsets("program") 

# An example of a synset: 
print(syns[0].name()) 

# Just the word: 
print(syns[0].lemmas()[0].name()) 

# Definition of that first synset: 
print(syns[0].definition()) 

# Examples of the word in use in sentences: 

print(syns[0].examples()) 
import nltk 
from nltk.corpus import wordnet 
synonyms = [] 
antonyms = [] 

for syn in wordnet.synsets("good"): 
    for l in syn.lemmas(): 
        synonyms.append(l.name()) 
        if l.antonyms(): 
            antonyms.append(l.antonyms()[0].name()) 

    print(set(synonyms)) 
    print(set(antonyms)) 



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


plan.n.01
plan
a series of steps to be carried out or goals to be accomplished
['they drew up a six-step plan', 'they discussed plans for a new bond issue']
{'good'}
set()
{'goodness', 'good'}
{'evilness', 'evil'}
{'goodness', 'good'}
{'evilness', 'bad', 'badness', 'evil'}
{'commodity', 'trade_good', 'goodness', 'good'}
{'evilness', 'bad', 'badness', 'evil'}
{'commodity', 'trade_good', 'goodness', 'good'}
{'evilness', 'bad', 'badness', 'evil'}
{'commodity', 'full', 'trade_good', 'goodness', 'good'}
{'evilness', 'bad', 'badness', 'evil'}
{'commodity', 'full', 'trade_good', 'goodness', 'good'}
{'evilness', 'bad', 'badness', 'evil'}
{'trade_good', 'goodness', 'good', 'commodity', 'full', 'honorable', 'estimable', 'respectable'}
{'evilness', 'bad', 'badness', 'evil'}
{'trade_good', 'goodness', 'good', 'commodity', 'full', 'honorable', 'estimable', 'beneficial', 'respectable'}
{'evilness', 'bad', 'badness', 'evil'}
{'trade_good', 'goodness', 'good', 'commodity', 'full', 'honorable', 'estima